#Gold Layer Scripting : Business-Level Data Modeling

This script creates views for the Gold layer in the data lakehouse. The Gold layer represents the final dimension and fact tables.Each view performs transformations and combines data from the Silver layer to produce a clean, enriched, and business-ready dataset, that answer real business questions.

### The Transformation Logic :
Create a 360° Products Dimension table which consolidates product data from CRM and ERP systems into a unified, analytics-ready structure. It enriches product records with category information to support reporting, segmentation, and performance analysis.

In [0]:
querry = """

select 
  row_number() over (order by cu.start_date , cu.product_number) as product_key ,
  cu.product_id ,
  cu.product_number ,
  cu.product_name ,
  cu.product_cost ,
  cu.product_line ,
  pc.category ,
  pc.subcategory ,
  pc.maintenance_flag ,
  cu.start_date 


from
  bike_lakehouse.silver.crm_products as cu

left join
  bike_lakehouse.silver.erp_product_category as pc

on
  cu.category_id = pc.category_id

-- Filter out all historical data
where
  cu.end_date is  null

  """

In [0]:
df = spark.sql(querry)

In [0]:
df.limit(10).display()

product_key,product_id,product_number,product_name,product_cost,product_line,category,subcategory,maintenance_flag,start_date
1,210,FR-R92B-58,HL Road Frame - Black- 58,0,Road,Components,Road Frames,true,2003-07-01
2,211,FR-R92R-58,HL Road Frame - Red- 58,0,Road,Components,Road Frames,true,2003-07-01
3,601,BB-7421,LL Bottom Bracket,24,n/a,Components,Bottom Brackets,true,2013-07-01
4,602,BB-8107,ML Bottom Bracket,45,n/a,Components,Bottom Brackets,true,2013-07-01
5,603,BB-9108,HL Bottom Bracket,54,n/a,Components,Bottom Brackets,true,2013-07-01
6,478,BC-M005,Mountain Bottle Cage,4,Mountain,Accessories,Bottles and Cages,false,2013-07-01
7,479,BC-R205,Road Bottle Cage,3,Road,Accessories,Bottles and Cages,false,2013-07-01
8,596,BK-M18B-40,Mountain-500 Black- 40,295,Mountain,Bikes,Mountain Bikes,true,2013-07-01
9,597,BK-M18B-42,Mountain-500 Black- 42,295,Mountain,Bikes,Mountain Bikes,true,2013-07-01
10,598,BK-M18B-44,Mountain-500 Black- 44,295,Mountain,Bikes,Mountain Bikes,true,2013-07-01


### Writing Gold Table
Persist the cleaned DataFrame as a Delta table in the Gold layer.

In [0]:
df.write.mode("overwrite").format('delta').saveAsTable('bike_lakehouse.gold.dim_products')

### Sanity checks of Gold table
Quickly check the result of transformations, before moving forward with the dataFrame

In [0]:
%sql
select * from bike_lakehouse.gold.dim_products limit 10;

product_key,product_id,product_number,product_name,product_cost,product_line,category,subcategory,maintenance_flag,start_date
1,210,FR-R92B-58,HL Road Frame - Black- 58,0,Road,Components,Road Frames,true,2003-07-01
2,211,FR-R92R-58,HL Road Frame - Red- 58,0,Road,Components,Road Frames,true,2003-07-01
3,601,BB-7421,LL Bottom Bracket,24,n/a,Components,Bottom Brackets,true,2013-07-01
4,602,BB-8107,ML Bottom Bracket,45,n/a,Components,Bottom Brackets,true,2013-07-01
5,603,BB-9108,HL Bottom Bracket,54,n/a,Components,Bottom Brackets,true,2013-07-01
6,478,BC-M005,Mountain Bottle Cage,4,Mountain,Accessories,Bottles and Cages,false,2013-07-01
7,479,BC-R205,Road Bottle Cage,3,Road,Accessories,Bottles and Cages,false,2013-07-01
8,596,BK-M18B-40,Mountain-500 Black- 40,295,Mountain,Bikes,Mountain Bikes,true,2013-07-01
9,597,BK-M18B-42,Mountain-500 Black- 42,295,Mountain,Bikes,Mountain Bikes,true,2013-07-01
10,598,BK-M18B-44,Mountain-500 Black- 44,295,Mountain,Bikes,Mountain Bikes,true,2013-07-01
